In [6]:
import os
import glob
import requests
import ast
import threading

import pandas as pd

from utils import get_output_path

In [7]:
CITY = 'singapore'
# CITY = 'hanoi'

OVERPASS_API_URL = "http://localhost:8000/api/interpreter"
DATA_DIR = f"../data/{CITY}"

In [8]:
def call_overpass_api(nodes, lat, lon):
    node_1, node_2 = nodes
    overpass_query = f"""
        [out:json];
        node(id:{node_1}, {node_2});
        way(bn);
        out count tags;
    """

    response = requests.get(
        OVERPASS_API_URL, 
        params={'data': overpass_query}
    )
    data = response.json()
    
    tags, ways = [], []
    for element in data['elements']:
        ways.append(element['id'])
        if 'tags' in element:
            tags.append(element['tags'])

    info = [(
        lat,
        lon,
        ways,
        tags,
    )]

    df = pd.DataFrame(info, columns=[
        'lat',
        'lon',
        'ways',
        'tags',
    ])

    return df

def query_overpass_api(df):
    results = []

    for _, row in df.iterrows():
        overpass_df = call_overpass_api(row['nodes'], row['lat'], row['lon']) 
        results.append(overpass_df)

    return pd.concat(results).reset_index(drop=True)

def process_file(csv_file):
    print(f"Processing: {csv_file}")

    output_path = get_output_path(f"{DATA_DIR}/ways", csv_file, 'csv')
    if os.path.exists(output_path):
        return
    
    df = pd.read_csv(csv_file)
    df['nodes'] = df['nodes'].apply(ast.literal_eval)

    overpass_df = query_overpass_api(df)
    overpass_df.to_csv(output_path, index=False)

def get_ways():    
    csv_files = glob.glob(
        os.path.join(f"{DATA_DIR}/nodes", '*.csv')
    )
    for csv_file in csv_files:
        thread = threading.Thread(target=process_file, args=(csv_file,))
        thread.start()

In [9]:
get_ways()

Processing: ../data/singapore/nodes/Smash 80km.csv
Processing: ../data/singapore/nodes/Wild Boar Trail Run 25km.csv
Processing: ../data/singapore/nodes/HARDCORE 103km ver 2.csv
Processing: ../data/singapore/nodes/Wild Boar Trail Run 45km.csv
Processing: ../data/singapore/nodes/Shiok! 200km.csv
Processing: ../data/singapore/nodes/Destroy 80km.csv
Processing: ../data/singapore/nodes/Lucky 88.88km 2024.csv
Processing: ../data/singapore/nodes/Gila100km.csv
Processing: ../data/singapore/nodes/RouteACrazeUltra101km.csv
Processing: ../data/singapore/nodes/SG200Miles2024.csv
Processing: ../data/singapore/nodes/POWER La! 100km Ultramarathon.csv
Processing: ../data/singapore/nodes/100MilesChallenge.csv
